In [1]:
# !pip install transformers==v4.39.2
# !pip uninstall -y transformers
# !pip install git+https://github.com/huggingface/transformers
# !pip install -q pillow accelerate einops


In [2]:
!pip show transformers

Name: transformers
Version: 4.39.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/envs/imp/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
# torch.set_default_device("cuda")
default_device = torch.device("cuda:0")
torch.cuda.set_device(default_device)
torch.__version__


/opt/conda/envs/imp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'2.0.1+cu117'

In [4]:
# !git clone https://huggingface.co/MILVLG/imp-v1-3b

In [5]:
#Create model

model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b", 
    torch_dtype=torch.float16, 
    device_map="auto",
    trust_remote_code=True,
    # force_download=True,
    resume_download=True)
tokenizer = AutoTokenizer.from_pretrained("MILVLG/imp-v1-3b", trust_remote_code=True)


/opt/conda/envs/imp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-05-20 14:45:05,422] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/envs/imp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  4.58it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import os

# for detection
text_car = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
        USER: <image>\nIs there any cars,persons,traffic lights,road signals, motorcycles in the image, \
            give me their bounding box coordinates in the image, just these 5 classes, each object must has 4 coordinates, \
            if there is no this kand of object, just return None? \
            Give me the answer in this format : \
                {'cars': [[x_min,y_min, x_max,y_max], [x_min,y_min,x_max,y_max], ..., [x_min,y_min,x_max,y_max]], \
                'persons': [[x_min,y_min, x_max,y_max], [x_min,y_min,x_max,y_max], ..., [x_min,y_min,x_max,y_max]], \
                    'traffic lights': [[x_min,y_min, x_max,y_max], [x_min,y_min,x_max,y_max], ..., [x_min,y_min,x_max,y_max]], \
                        'road signals': [[x_min,y_min, x_max,y_max], [x_min,y_min,x_max,y_max], ..., [x_min,y_min,x_max,y_max]], \
                            'motorcycles': [[x_min,y_min, x_max,y_max], [x_min,y_min,x_max,y_max], ..., [x_min,y_min,x_max,y_max]],} ASSISTANT:"

text1 = """
Imagine a friendly chat between an inquisitive user and an AI assistant, where the assistant is eager to provide helpful and detailed responses to the user's questions.

USER: [Attaches an image]
Could you please assist me in identifying any cars, people, traffic lights, road signals, or motorcycles in this image? Just these 5 classes, If any of these objects are present, could you kindly provide their bounding box coordinates within the image? For each of these five categories, please include four coordinates representing the minimum and maximum values for both the x and y axes. If any category of object is not detected, a placeholder like [] should be given.

The desired response format is as follows:
{
    'cars': [[x_min, y_min, x_max, y_max], ...],
    'persons': [[x_min, y_min, x_max, y_max], ...],
    'traffic lights': [[x_min, y_min, x_max, y_max], ...],
    'road signals': [[x_min, y_min, x_max, y_max], ...],
    'motorcycles': [[x_min, y_min, x_max, y_max], ...]
} 

ASSISTANT:

"""

text2 = """
Scene: A friendly interaction between an inquisitive user and an AI assistant who specializes in image analysis.

User: I've got an image here <image>, could you tell me if there are any cars, people, traffic lights, road signs, or motorcycles in it? \
    If there are, I need the bounding box coordinates for each, defined by four points. Just stick to these five categories, please. \
        Oh, and if there's nothing of that sort in the picture, just let me know by saying 'None'. I'd appreciate it if you could format your response like this:

```json
{
  "cars": [
    [x_min, y_min, x_max, y_max],
    [x_min, y_min, x_max, y_max],
    ...
  ],
  "persons": [
    [x_min, y_min, x_max, y_max],
    ...
  ],
  "traffic_lights": [
    ...
  ],
  "road_signals": [
    ...
  ],
  "motorcycles": [
    ...
  ]
}
```
"""

text_persons = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
        USER: <image>\nIs there any persons in the image, give me their bounding box coordinates in the image, each person must has 4 cordinates, if there is no person, just return None? \
            Give me the answer in this format : {'person1': [x_min,y_min, x_max,y_max], 'person2': [x_min,y_min,x_max,y_max], ...} ASSISTANT:"
text_traffic_lights = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
        USER: <image>\nIs there any traffic lights in the image, give me their bounding box coordinates in the image, each traffic light must has 4 cordinates, if there is no traffic light, just return None? \
            Give me the answer in this format : {'traffic_light1': [x_min,y_min, x_max,y_max], 'traffic_light2': [x_min,y_min,x_max,y_max], ...} ASSISTANT:"
text_road_signals = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
        USER: <image>\nIs there any road signals in the image, give me their bounding box coordinates in the image, each road signal must has 4 cordinates, if there is no road signal, just return None? \
            Give me the answer in this format : {'road_signal1': [x_min,y_min, x_max,y_max], 'road_signal2': [x_min,y_min,x_max,y_max], ...} ASSISTANT:"
text_motorcycles = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
        USER: <image>\nIs there any motorcycles in the image, give me their bounding box coordinates in the image, each motorcycle must has 4 cordinates, if there is no motorcycle, just return None? \
            Give me the answer in this format : {'motorcycle1': [x_min,y_min, x_max,y_max], 'motorcycle2': [x_min,y_min,x_max,y_max], ...} ASSISTANT:"
import json
from tqdm import tqdm

img_path = "/home/data/images"
img_lst = os.listdir(img_path)
img_lst = sorted(img_lst, key=lambda x: int(x.split('.')[0]))
out_dir = img_path.replace("images", "bboxes")
os.makedirs(out_dir, exist_ok=True)
for e in tqdm(img_lst):
    bbox = {}
    print(f"processing {e}")
    file_pth = os.path.join(img_path, e)
    image0 = Image.open(file_pth) # 000460
    input_ids = tokenizer(text1, return_tensors='pt').input_ids.to(default_device)
    image_tensor0 = model.image_preprocess(image0).to(default_device)

    #Generate the answer
    output_ids = model.generate(
        input_ids,
        max_new_tokens=500,
        images=image_tensor0,
        use_cache=True)[0]
    car_pos = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()
    try:
      bbox[e] = eval(car_pos)
    except:
       import pdb;pdb.set_trace()
    with open(os.path.join(out_dir, e.replace(".jpg",".json")),"w") as json_file:
        json.dump(bbox, json_file, indent=4)
    # print(car_pos)
# print(bbox)

In [20]:
import json
json_dir = "/home/data/bboxes"
js_list = os.listdir(json_dir)
with open(js_list[0],"r") as json_file:
    jf = json.load(json_file)

obj_bbox = jf[js_list[0].replace(".josn", ".jpg")]

for k,v in obj_bbox.items():
    

NameError: name 'bbox' is not defined

In [25]:
# car_pos = [[0.34, 0.38, 0.64, 0.82], [0.68, 0.51, 1.0, 0.74],[0.68, 0.51, 1.0, 0.74]]
h, w = 1028, 1912
car_pos_wh = []
for car in eval(car_pos).values():
    car_pos_wh.append([float(item) * w if idx % 2 == 0 else float(item) * h for idx, item in enumerate(car)])
print(car_pos_wh)

import cv2
import numpy as np
import matplotlib.pyplot as plt

# image = np.array(image0)
# Convert RGB to BGR (OpenCV uses BGR format)
# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image = cv2.imread(file)
for point in car_pos_wh:
    print(point)
    # Draw the box on the image
    color = (0, 255, 0)  # Green color in BGR
    thickness = 2  # Line thickness
    cv2.rectangle(image, (int(point[0]), int(point[1])), (int(point[2]), int(point[3])), color, thickness)


# Display the image with bounding boxes
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for Matplotlib
plt.axis('off')  # Hide axes
plt.show()

TypeError: float() argument must be a string or a real number, not 'list'

In [ ]:
# Yes, there is a car, a motorcycle, a person, traffic lights, and road signals in the image. for attacks
# Yes, there is a car driving down the street, a traffic light, and a road signal in the image. for ori

In [ ]:
inputs_embeds = model.get_input_embeddings()(input_ids)
inputs_embeds

In [ ]:
img_f = model.encode_images(image_tensor.to(torch.float16))
# img_f.shape # torch.Size([1, 729, 2560])
model.en

In [ ]:
outputs = model(
    input_ids=input_ids,
    inputs_embeds=inputs_embeds,
    images=image
)

# Extract the relevant features and embeddings from the outputs
text_features = outputs.hidden_states
# outputs.logits.shape # torch.Size([1, 49, 51200])
text_features

In [ ]:
s = model.generate(
    input_ids,
    max_new_tokens=100,
    images=image_tensor,
    use_cache=True)

In [ ]:
s[0]